Importiamo il nostro equipaggiamento


In [1]:
import sqlite3, time, csv, re, random, string
import time
import codecs
import numpy as np
import sqlite3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from langdetect import detect, DetectorFactory
from scipy.sparse import csr_matrix
from sklearn.metrics import classification_report
DetectorFactory.seed = 0
import warnings
warnings.filterwarnings("ignore") 

# 1. Pulizia post (che io ho fatto una volta, su 10 000 sarcastici e 10 000 non sarcastici e poi sempre letto da file)

Connetto al database SQL e scarico i post in due diversi dataframe, uno per i post sarcastici, uno per quelli "seri"

In [2]:
# sql_conn = sqlite3.connect('C:\\Users\\Flavio Macci\\Desktop\\database.sqlite')
sql_conn = sqlite3.connect('/home/IT/Documenti/ML/reddit-comments-may-2015/database.sqlite')

In [27]:
df_sarc = pd.read_sql("SELECT ups, name, subreddit, id, gilded, downs, score, body, distinguished, controversiality, parent_id FROM May2015 WHERE LENGTH(body) > 20 AND LENGTH(body) < 200 AND body LIKE '% /s' LIMIT 50000", sql_conn)

SyntaxError: EOL while scanning string literal (<ipython-input-27-e37e8fcba648>, line 1)

In [5]:
df_ser = pd.read_sql("SELECT ups, name, subreddit, id, gilded, downs, score, body, distinguished, controversiality, parent_id FROM May2015 WHERE LENGTH(body) > 20 AND LENGTH(body) < 200 AND body NOT LIKE '% /s' LIMIT 25000", sql_conn)

Pulisco i post sarcastici, rimuovendo i label " /s"

In [6]:
df_sarc['body'].replace(value = '', regex = ' /s', inplace = True)

In [7]:
df_sarc['is_sarc'] = [1]*len(df_sarc)
df_ser['is_sarc'] = [0]*len(df_ser)

In [8]:
def cleaner(text):
    text = ''.join(i for i in text if ord(i)<128 and not i.isdigit())
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',text)
    text = re.sub("\n"," ",text)
    text = re.sub("\r"," ",text)
    text = ''.join(i for i in text if not i.isdigit())
    return text

In [9]:
data = shuffle(df_ser.append(df_sarc)).reset_index(drop = True)

In [10]:
data['body'] = data['body'].apply(cleaner)

In [11]:
for i in range(0, len(data)):
    try:
        if detect(data["body"][i]) != "en":
            data.drop(i, inplace = True)
    except:
        data.drop(i, inplace = True)

In [12]:
df = data.drop(["name","id","parent_id","ups","downs", "score",
                          "gilded","distinguished","controversiality","parent_id"], axis = 1, inplace = True)

# 2. Analisi

Si lavora su data (no punteggiatura no lingue straniere)

Dopo la prima volta runna da qui che data clean lo legge direttamente da file pulito (no lingue straniere nè punteggiatura)

In [2]:
data = pd.read_csv('data_50000')

In [3]:
data = data[: 30000]

Da qui all'inizio del prossimo sottocapitolo è codice inutile che serve solo perchè ho letto da file

In [4]:
data.isnull().sum()

Unnamed: 0    0
subreddit     0
body          0
is_sarc       0
dtype: int64

In [5]:
data.dropna(inplace = True)

## Clean

Logistic regression

In [6]:
margin= int(4/5*len(data))

In [7]:
text_train= data[:margin]
text_test = data[margin:]
y_train = text_train["is_sarc"]
y_test = text_test["is_sarc"]
text_train = text_train.drop("is_sarc", axis = 1)
text_test = text_test.drop("is_sarc", axis = 1)

Bag of words con ngarms = (1, 2) ) (quindi oltre alle parole ti considera anche le coppie)

In [8]:
vect = CountVectorizer(min_df = 5, stop_words = "english", ngram_range = (1, 2)).fit(text_train["body"])
X_train = vect.transform(text_train["body"])

Il prossimo comando seleziona le variabili utili

In [9]:
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(MultinomialNB(), param_grid, cv = 5)
grid.fit(X_train, y_train)
words_selector = SelectFromModel(grid.best_estimator_, threshold = 10**(-1)).fit(X_train, y_train)
useful_words = [list(vect.vocabulary_.keys())[i] for i in words_selector.get_support(indices = True)]

Questo invece mostra la performance senza subreddit

In [10]:
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(MultinomialNB(), param_grid, cv=10)
grid.fit(X_train, y_train)
X_test = vect.transform(text_test["body"])
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.68      0.69      0.68      2916
          1       0.70      0.69      0.70      3084

avg / total       0.69      0.69      0.69      6000



## Pulito con subreddit

Aggiungo un campo text contentente gli ngrammi significativi selezionati al paragrafo precedente seguiti dal corrispondente subreddit (sotto ho stampato un esempio)

In [11]:
def concat_subreddit(row) :
    list_of_words = row['body'].split(' ')
    list_of_words = map(lambda word : word + ' ' + row['subreddit'] if word in useful_words else '', list_of_words)
    return ' '.join(list_of_words)

In [12]:
data['body_subreddit'] = data[['body', 'subreddit']].apply(concat_subreddit, axis = 1)

In [13]:
data['text'] = data['body'] + ' ' + data['body_subreddit']

Example

In [14]:
print('body =', data['body'][2])
print('subreddit =', data['subreddit'][2])
print('text = ', data['text'][2])

body = This is Carlsbad   It is much more conservative   So it is scarier still  
subreddit = OkCupid
text =  This is Carlsbad   It is much more conservative   So it is scarier still            conservative OkCupid         


Adesso text viene vettorizzato con ngrams = (1, 3) per mettere nel vocabolario anche parole e coppie seguite dal subreddit (quindi, ad esempio, è come se stessimo considerando, nell'esempio, tutte le parole del body, più really nel contesto 'AskReddit' e surprising nel contesto 'AskReddit' e, se il 2gramma really surprising fosse stato nella lista degli ngrammi utili, avremmo considerato anche really surprising AskReddit, ovvero l'espressione really surprising nel contesto AskReddit)

In [15]:
margin = int(4/5 * len(data))

In [16]:
text_train = data[:margin]
text_test = data[margin:]
y_train = text_train["is_sarc"]
y_test = text_test["is_sarc"]
text_train = text_train.drop("is_sarc", axis = 1)
text_test = text_test.drop("is_sarc", axis = 1)

In [17]:
vect = CountVectorizer(min_df = 5, stop_words = "english", ngram_range = (1, 3)).fit(text_train["text"])
X_train = vect.transform(text_train["text"])

In [18]:
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(MultinomialNB(), param_grid, cv=10)
grid.fit(X_train, y_train)
X_test = vect.transform(text_test["text"])
y_pred = grid.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.73      0.74      0.73      2916
          1       0.75      0.74      0.74      3084

avg / total       0.74      0.74      0.74      6000

